In [69]:
#导入需要的模块
import numpy as np
import pandas as pd
import math
#from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib notebook

In [70]:
#数据读取
train_set_1 = pd.read_csv('train_hf_data.csv',header=0,index_col=0)
train_set_1_label = pd.read_csv('train_hf_label.csv',header=0,index_col=0)
test_set_1 = pd.read_csv('test_hf_data.csv',header=0,index_col=0)
test_set_1_label = pd.read_csv('test_hf_label.csv',header=0,index_col=0)

In [71]:
#设计函数清理不在给定范围内的样本
def data_clean(data):
    D = data
    D = D.drop(D[(D.age < 40) | (D.age > 95)].index)
    D = D.drop(D[(D.creatinine_phosphokinase < 23) | (D.creatinine_phosphokinase > 7861)].index)
    D = D.drop(D[(D.ejection_fraction < 14) | (D.ejection_fraction > 80)].index)
    D = D.drop(D[(D.platelets < 25010) | (D.platelets > 850000)].index)
    D = D.drop(D[(D.serum_creatinine < 0.5) | (D.serum_creatinine > 9.4)].index)
    D = D.drop(D[(D.serum_sodium < 114) | (D.serum_sodium > 148)].index)
    D = D.drop(D[(D.time < 4) | (D.time > 285)].index)
    return D.reset_index(drop=True)

In [72]:
#设计函数将数据按附件给的正常范围进行归一化处理
#这个函数是用的老师给的数据范围进行归一化的
def normalization(data):
    D = np.array(data)
    min_range = np.array([[40,55],[0,1],[23,7838],[0,1],[14,66],[0,1],[25010,825000],[0.5,8.9],[114,34],[0,1],[0,1],[4,281]])
    def maxminNormalize(X,X_min,X_range):
        return (X-X_min) / X_range
    for i in range(len(D[0]-1)):
        for j in range(len(D[:,0]-1)):
            D[j][i] = maxminNormalize(D[j][i],min_range[i][0],min_range[i][1])
    return D     

In [73]:
#对训练集清洗
train_set_1_merge = pd.concat([train_set_1,train_set_1_label],axis=1)#为了数据清洗方便，先将标签合并上去
train_set_1_merge = data_clean(train_set_1_merge)#清除不在范围的样本及标签
#为了便于运算，重新创建两个dataframe，data_0只有标签为0的数据；data_1只有标签为1的数据
data_0 = train_set_1_merge
data_0 = data_0.drop(data_0[data_0.label == 1].index).reset_index(drop=True)
data_1 = train_set_1_merge
data_1 = data_1.drop(data_1[data_1.label == 0].index).reset_index(drop=True)
y_0 = math.log(len(data_1['label']) / len(data_0['label']),10)#计算判定时需要的阈值
#删除标签
data_0 = data_0.drop('label',axis=1)
data_1 = data_1.drop('label',axis=1)
data_0 = normalization(data_0)#标准化标签为0的数据
m_0 = np.mean(data_0,axis=0)#计算标签为0的均值向量
s_0 = np.zeros(shape=(len(m_0),len(m_0)))#定义标签为0的类内离散度矩阵
for i in range(len(data_0)-1):
    s_0 += np.dot((data_0[i]-m_0).reshape(12,1),(data_0[i]-m_0).reshape(1,12))#计算标签为0的类内离散度矩阵
data_1 = normalization(data_1)#标准化标签为1的数据
m_1 = np.mean(data_1,axis=0)#计算标签为1的均值向量
s_1 = np.zeros(shape=(len(m_0),len(m_0)))#定义标签为1的类内离散度矩阵
for i in range(len(data_1)-1):
    s_1 += np.dot((data_1[i]-m_0).reshape(12,1),(data_1[i]-m_0).reshape(1,12))#计算标签为1的类内离散度矩阵
s = np.mat(s_0 + s_1)#计算总类内离散度矩阵
w = np.dot(s.I,(m_0 - m_1))#计算参数矩阵
print('运算出的参数矩阵为:',w)
#以下为验证训练集的收敛情况
count_0 = 0
for i in range(len(data_0)-1):
    if np.dot(w,data_0[i]-1/2*(m_0 + m_1)) <= y_0:
        count_0 += 1
print('训练集中标签为0的误分类个数：',count_0)
count_1 = 0
for i in range(len(data_1)-1):
    if np.dot(w,data_1[i]-1/2*(m_0 + m_1)) >= y_0:
        count_1 += 1
print('训练集中标签为1的误分类个数：',count_1)
#对测试集清洗
test_set_1_merge = pd.concat([test_set_1,test_set_1_label],axis=1)#为了数据清洗方便，先将标签合并上去
test_set_1_merge = data_clean(test_set_1_merge)#清除不在范围的样本及标签
#对测试集进行相同的操作
test_0 = test_set_1_merge
test_0 = test_0.drop(test_0[test_0.label == 1].index).reset_index(drop=True)
test_1 = test_set_1_merge
test_1 = test_1.drop(test_1[test_1.label == 0].index).reset_index(drop=True)
#删除标签
test_0 = test_0.drop('label',axis=1)
test_1 = test_1.drop('label',axis=1)
test_0 = normalization(test_0)
test_1 = normalization(test_1)
#计算测试集中的误分类个数，并输出错误率
count_0 = 0
for i in range(len(test_0)-1):
    if np.dot(w,test_0[i]-1/2*(m_0 + m_1)) <= y_0:
        count_0 += 1
print('测试集中标签为0的误分类个数：',count_0)
count_1 = 0
for i in range(len(test_1)-1):
    if np.dot(w,test_1[i]-1/2*(m_0 + m_1)) >= y_0:
        count_1 += 1
print('测试集中标签为1的误分类个数：',count_1)
print("Error rate: ",(count_0 + count_1) / len(test_set_1_merge['label']))#

运算出的参数矩阵为: [[-0.00549126  0.00046238 -0.00393476 -0.00062357  0.01064647  0.00086281
   0.00301802 -0.01076436  0.00561992  0.00034739 -0.00018436  0.01238021]]
训练集中标签为0的误分类个数： 0
训练集中标签为1的误分类个数： 74
测试集中标签为0的误分类个数： 0
测试集中标签为1的误分类个数： 20
Error rate:  0.3389830508474576
